In [ ]:
%pip install cron-descriptor
dbutils.library.restartPython()

In [0]:
from cron_descriptor import get_description, Options
from pyspark.sql import SparkSession as SparkSession, functions as F, types as T
import pandas as pd

opts = Options()
# opts.locale_code = "ja_JP"            # 日本語
opts.use_24hour_time_format = True    # 24時間表記

In [ ]:
LINE_SEP = "  \n"
SLACK_URL = "https://www.youtube.com/"

In [ ]:
def normalize_text(text:str | None) -> str:
    """ 文字列の整形(explanation、type_conversion、rule用)
    """
    if text is None:
        return ""
    return text.strip().replace("\n", LINE_SEP)


def generate_description_schema(name_ja:str, explanation:str) -> str:
    """ 下記の形式で説明文を生成する
    データソース名/システム名：{name_ja}
    Overview：このスキーマに含まれるデータは、主に上記のシステムから取り込まれています。
    データソース概要：{explanation}
    連絡先：[#sys-データ基盤の相談や情報共有](SLACK_URL)
    """
    return f"データソース名/システム名：{name_ja}{LINE_SEP}Overview：このスキーマに含まれるデータは、主に上記のシステムから取り込まれています。{LINE_SEP}データソース概要：{explanation}{LINE_SEP}連絡先：[#sys-データ基盤の相談や情報共有]({SLACK_URL})"


def generate_description_table(name_ja:str, explanation:str,type_conversion:str,rule:str,schedule:str,query:str,reference:str,link:str|None=None) -> str:
    """ 下記の形式で説明文を生成する
    TableName：{name_ja}
    Overview：
    {explanation}
    {type_conversion}
    {rule}
    更新頻度：{schedule}
    元テーブル仕様書：[link]({link})
    作成クエリ：[query]({query})
    対応チケット：[reference]({reference})
    連絡先：[#sys-データ基盤の相談や情報共有](SLACK_URL)
    """
    parts = [
        f"TableName：{name_ja}",
        "Overview：",
        explanation,
        type_conversion,
        rule,
        f"更新頻度：{schedule}",
    ]

    if link is not None:
        parts.append(f"元テーブル仕様書：[link]({link})")

    parts.extend([
        f"作成クエリ：[query]({query})",
        f"対応チケット：[reference]({reference})",
        f"連絡先：[#sys-データ基盤の相談や情報共有]({SLACK_URL})",
    ])
    return LINE_SEP.join(parts)


def generate_description_view(name_ja:str, schedule:str, reference:str):
    """ 下記の形式で説明文を生成する
    TableName：{name_ja}
    Overview：元テーブル名から必要なカラムを取り出したビュー。詳細を確認したい場合は、元テーブルを参照してください。
    更新頻度：{schedule}
    対応チケット：[reference]({reference})
    連絡先：[#sys-データ基盤の相談や情報共有]({SLACK_URL})
    """
    return f"TableName：{name_ja}{LINE_SEP}Overview：元テーブル名から必要なカラムを取り出したビュー。詳細を確認したい場合は、元テーブルを参照してください。{LINE_SEP}更新頻度：{schedule}{LINE_SEP}対応チケット：[reference]({reference}){LINE_SEP}連絡先：[#sys-データ基盤の相談や情報共有]({SLACK_URL})"



In [ ]:
try:
    # 引数の受け取り
    dbutils.widgets.text("SOURCE_TABLE", "")
    source_table = dbutils.widgets.get("SOURCE_TABLE")
    dbutils.widgets.text("ADDITIONAL_WHERE_TEXT", "")
    additional_where_text = dbutils.widgets.get("ADDITIONAL_WHERE_TEXT")

    # SQL文の生成
    sql_text = f"""
    SELECT *
    FROM {source_table}
    WHERE TRUE
        {additional_where_text}
    """
    print(sql_text)
    spark = SparkSession.builder.getOrCreate()
    df_target = spark.sql(sql_text)
    # df_target = spark.table(source_table)

    display(df_target) 
except Exception as e:
    raise Exception(f"`{source_table}`テーブルの取得に失敗しました。SQL: \"{sql_text}\", Error: {e}")

In [ ]:
if df_target.count() == 0:
    print("Descriptionの更新対象がなかったため、処理をスキップして終了しました。")
else:
    error_records = []
    df_target_sorted = df_target.orderBy(F.col("catalog"), F.col("schema"), F.col("table"))
    for row in df_target_sorted.toLocalIterator():
        # カラムの取得
        catalog         = row['catalog']
        schema          = row['schema']
        table           = row['table']
        name_ja         = row['name_ja']
        table_category  = row['table_category']
        cron_schedule   = row['cron_schedule']
        explanation     = normalize_text(row['explanation'])
        type_conversion = normalize_text(row['type_conversion'])
        rule            = normalize_text(row['rule'])
        link            = row['link']
        query           = row['query']
        reference       = row['reference']

        # cron式の自然言語変換
        update_schedule_expr = None
        try:
            if cron_schedule is not None and cron_schedule != "":
                update_schedule_expr = get_description(cron_schedule, options=opts)
        except Exception as e:
            error_records.append({
                "catalog": catalog,
                "schema": schema,
                "table": table,
                "table_category": table_category,
                "error_category": "cron_schedule_error",
                "cron_schedule": cron_schedule,
                "sql": None,
                "error_message": str(e)
            })
            continue

        # Descriptionの生成
        description = ""
        try:
            if table_category == 0:
                # description += generate_description_schema(name_ja, explanation)
                raise ValueError(f"スキーマは、ジョブでの更新対象外です。手動で更新してください。table_category={table_category}")
            elif table_category == 1:
                if "staging" in catalog:
                    description += generate_description_table(name_ja, explanation, type_conversion, rule, update_schedule_expr, query, reference, link)
                elif "conformed" in catalog or "published" in catalog:
                    description += generate_description_table(name_ja, explanation, type_conversion, rule, update_schedule_expr, query, reference)
                else:
                    raise ValueError(f"カタログがジョブの更新対象外です。このジョブは「staging、conformed、published」を含むカタログのみを対象にしています。: catalog=\"{catalog}\"")
            elif table_category == 2:
                description += generate_description_view(name_ja, update_schedule_expr, reference)
            elif table_category == 3:
                raise ValueError(f"ビューテーブルは、ジョブでの更新対象外です。手動で更新してください。table_category={table_category}")
            else:
                raise ValueError(f"table_category is not 1 or 2: table_category={table_category}")
        except Exception as e:
            error_records.append({
                "catalog": catalog,
                "schema": schema,
                "table": table,
                "table_category": table_category,
                "error_category": "generate_description_error",
                "cron_schedule": cron_schedule,
                "sql": None,
                "error_message": str(e)
            })
            continue
        # print(description) # デバッグ用

        # SQLの実行
        sql = ""
        try:
            if table_category == 1: # Tableの場合
                sql += f"COMMENT ON TABLE {catalog}.{schema}.{table} IS '{description}'"
            elif table_category == 2: # Materialized Viewの場合
                sql += f"COMMENT ON VIEW {catalog}.{schema}.{table} IS '{description}'"
            else:
                raise ValueError(f"table_category is not 1 or 2: table_category={table_category}")
            spark.sql(sql)

        except Exception as e:
            error_records.append({
                "catalog": catalog,
                "schema": schema,
                "table": table,
                "table_category": table_category,
                "error_category": "update_description_error",
                "cron_schedule": cron_schedule,
                "sql": sql,
                "error_message": str(e)
            })

    # エラーがある場合、エラーを表示
    if len(error_records) > 0:
        df_error = pd.DataFrame(error_records).astype("string")
        display(df_error)
        raise Exception("Descriptionの更新に失敗したレコードがあります。")
    else:
        print("Descriptionの更新がすべて成功しました。")
